In [1]:
%load_ext autoreload
%autoreload 2

import json

from dac_costing.model import DacModel, DacSection, BatterySection, EnergySection

## Model Parameters

Here we open the default model parameters. We can modify these as needed...

In [2]:
with open('../dac_costing/data/parameters.json', 'r') as f:
    params = json.load(f)

In [3]:
from uncertainties import ufloat
stdev = 0.1

def cast_to_ufloat(d):
    u = {}
    for p, val in d.items():
        if isinstance(val, dict):
            u[p] = cast_to_ufloat(val)
        if isinstance(val, float):
            u[p] = ufloat(val, val*stdev, tag=p)
        else:
            u[p] = val
    return u

params = cast_to_ufloat(params)

## C1 - Natural Gas

In [4]:
from dac_costing.model import NgThermalSection

In [5]:
params['Base Energy Requierement [MW]'] = 46.6  # ='Report Data'!C58
electric = EnergySection('NGCC w/ CCS', battery=None, **params)
electric.compute().series

params['Required Thermal Energy [GJ/tCO2]'] = 6.64
thermal = NgThermalSection('Advanced NGCC', battery=None, **params)

params['Total Capex [$]'] = 1029  # =+'Report Data'!C21
dac = DacSection(**params)
dac_all = DacModel(electric, thermal, dac, **params)
dac_all.compute().series

Total Capital Cost [M$]          1479+/-24
Capital Recovery [$/tCO2eq]       156+/-12
Fixed O&M [$/tCO2eq]               36.3977
Variable O&M [$/tCO2eq]        6.14+/-0.21
Natural Gas Cost [$/tCO2]       30.5+/-3.2
Total Cost [$/tCO2]               229+/-13
dtype: object

## C2 - Electric Kiln (Solar)

In [6]:
params['Base Energy Requierement [MW]'] = 38  # ='Report Data'!C64
ebattery = BatterySection(**params)
electric = EnergySection('Solar', battery=ebattery, **params)

params['Base Energy Requierement [MW]'] = 234  # =F18
tbattery = BatterySection(**params)
thermal = EnergySection('Solar', battery=tbattery, **params)

params['Total Capex [$]'] = 936.01  # ='Report Data'!H27
dac = DacSection(**params)

dac_all = DacModel(electric, thermal, dac, **params)
dac_all.compute().series

Total Capital Cost [M$]        3759+/-32
Capital Recovery [$/tCO2eq]     397+/-28
Fixed O&M [$/tCO2eq]             72.2679
Variable O&M [$/tCO2eq]          4.0+/-0
Natural Gas Cost [$/tCO2]        0.0+/-0
Total Cost [$/tCO2]             473+/-28
dtype: object

## C3 - Electric Kiln (Nuclear)

In [7]:
params['Base Energy Requierement [MW]'] = 38  # ='Report Data'!C64
electric = EnergySection('Advanced Nuclear', battery=None, **params)

params['Base Energy Requierement [MW]'] = 234  # =F18
thermal = EnergySection('Advanced Nuclear', battery=None, **params)

params['Total Capex [$]'] = 936.01  # ='Report Data'!H27
dac = DacSection(**params)

dac_all = DacModel(electric, thermal, dac, **params)
dac_all.compute().series

Total Capital Cost [M$]        (6.39+/-0.17)e+03
Capital Recovery [$/tCO2eq]      (6.8+/-0.6)e+02
Fixed O&M [$/tCO2eq]                     109.415
Variable O&M [$/tCO2eq]                9.1+/-0.5
Natural Gas Cost [$/tCO2]                77+/-11
Total Cost [$/tCO2]              (8.7+/-0.6)e+02
dtype: object

## C4 - Electric Kiln (Solar)

In [8]:
params['Base Energy Requierement [MW]'] = 38  # ='Report Data'!C64
ebattery = BatterySection(**params)
electric = EnergySection('Wind', battery=ebattery, **params)

params['Base Energy Requierement [MW]'] = 234  # =F18
tbattery = BatterySection(**params)
thermal = EnergySection('Wind', battery=tbattery, **params)

params['Total Capex [$]'] = 936.01  # ='Report Data'!H27
dac = DacSection(**params)

dac_all = DacModel(electric, thermal, dac, **params)
dac_all.compute().series

Total Capital Cost [M$]        3045+/-31
Capital Recovery [$/tCO2eq]     322+/-23
Fixed O&M [$/tCO2eq]             67.8671
Variable O&M [$/tCO2eq]          4.0+/-0
Natural Gas Cost [$/tCO2]        0.0+/-0
Total Cost [$/tCO2]             394+/-23
dtype: object